In [204]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dbfread import DBF
from haversine import haversine
from tqdm.notebook import tqdm
%matplotlib inline
import platform
import pickle
import re
platform.system()
if platform.system() == 'Darwin': # Mac 환경 폰트 설정
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows': # Windows 환경 폰트 설정
    plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False) # 마이너스 폰트 설정
%config InlineBackend.figure_format = 'retina'
!pip install dbfread
!pip install haversine

# 월별 승하차 총인원 데이터로 유사도 구하기

### 라벨링된 버스 총승하차 월별시간별

In [205]:
bus_train_num_1 = pd.read_csv('./2002버스지하철매칭 월별 승하차 합계.csv', index_col=0)
bus_train_num_2 = pd.read_csv('./2003버스지하철매칭 월별 승하차 합계.csv', index_col=0)
bus_train_num_3 = pd.read_csv('./2004버스지하철매칭 월별 승하차 합계.csv', index_col=0)
bus_train_num_4 = pd.read_csv('./2005버스지하철매칭 월별 승하차 합계.csv', index_col=0)

In [206]:
bus_train_num_1

,표준버스정류장ID,역명,사용년월,00시승차총승객수,00시하차총승객수,1시승차총승객수,1시하차총승객수,2시승차총승객수,2시하차총승객수,3시승차총승객수,...,19시하차총승객수,20시승차총승객수,20시하차총승객수,21시승차총승객수,21시하차총승객수,22시승차총승객수,22시하차총승객수,23시승차총승객수,23시하차총승객수,지하철역
0,100000001,종로2가사거리,202002,157,190,190,450,126,177,357,...,813,577,457,633,312,483,314,287,168,종로3가(탑골공원)
1,100000002,창경궁.서울대학교병원,202002,41,60,0,3,0,0,0,...,510,1228,428,1074,477,899,388,502,164,혜화
2,100000003,명륜3가.성대입구,202002,413,180,22,23,0,0,0,...,2434,4814,1705,5656,1332,6523,1270,2762,637,혜화
3,100000004,종로2가.삼일교,202002,82,19,182,90,75,33,115,...,780,1323,592,1523,590,1330,438,564,165,을지로3가
4,100000005,혜화동로터리.여운형활동터,202002,130,270,7,37,0,0,0,...,3207,1288,1773,1449,1587,1536,1562,463,793,한성대입구(삼선교)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10665,124900120,고덕1동주민센터.두레근린공원,202002,2,39,0,0,0,0,0,...,610,33,398,15,377,25,422,18,281,고덕
10666,124900121,고덕역1번출구,202002,0,0,0,0,0,0,0,...,252,107,158,81,127,59,97,33,35,고덕
10667,124900122,고덕역1번출구,202002,82,24,0,0,0,0,0,...,297,586,210,502,192,617,189,540,127,고덕
10668,124900123,명일신동아아파트,202002,0,0,0,0,0,0,0,...,84,412,47,485,73,560,46,212,22,고덕


In [207]:
#col:123을 위함, col_:4를 위함
up_col = ['역명', '00시승차총승객수', '1시승차총승객수','2시승차총승객수', '3시승차총승객수', '4시승차총승객수', '5시승차총승객수',  '6시승차총승객수', '7시승차총승객수','8시승차총승객수',  '9시승차총승객수',  '10시승차총승객수','11시승차총승객수',  '12시승차총승객수','13시승차총승객수', '14시승차총승객수',  '15시승차총승객수','16시승차총승객수', '17시승차총승객수','18시승차총승객수', '19시승차총승객수', '20시승차총승객수','21시승차총승객수',  '22시승차총승객수', '23시승차총승객수',  '지하철역']
down_col = ['역명', '00시하차총승객수', '1시하차총승객수','2시하차총승객수', '3시하차총승객수', '4시하차총승객수', '5시하차총승객수',  '6시하차총승객수', '7시하차총승객수','8시하차총승객수',  '9시하차총승객수',  '10시하차총승객수','11시하차총승객수',  '12시하차총승객수','13시하차총승객수', '14시하차총승객수',  '15시하차총승객수','16시하차총승객수', '17시하차총승객수','18시하차총승객수', '19시하차총승객수', '20시하차총승객수','21시하차총승객수',  '22시하차총승객수', '23시하차총승객수',  '지하철역']
up_col_ = ['역명', '00시승차총승객수', '1시승차총승객수','2시승차총승객수', '3시승차총승객수', '4시승차총승객수', '5시승차총승객수',  '6시승차총승객수', '7시승차총승객수','8시승차총승객수',  '9시승차총승객수',  '10시승차총승객수','11시승차총승객수',  '12시승차총승객수','13시승차총승객수', '14시승차총승객수',  '15시승차총승객수','16시승차총승객수', '17시승차총승객수','18시승차총승객수', '19시승차총승객수', '20시승차총승객수','21시승차총승객수',  '22시승차총승객수', '23시승차총승객수',  '지하철역']
down_col_ = ['역명', '00시하차총승객수', '1시하차총승객수','2시하차총승객수', '3시하차총승객수', '4시하차총승객수', '5시하차총승객수',  '6시하차총승객수', '7시하차총승객수','8시하차총승객수',  '9시하차총승객수',  '10시하차총승객수','11시하차총승객수',  '12시하차총승객수','13시하차총승객수', '14시하차총승객수',  '15시하차총승객수','16시하차총승객수', '17시하차총승객수','18시하차총승객수', '19시하차총승객수', '20시하차총승객수','21시하차총승객수',  '22시하차총승객수', '23시하차총승객수',  '지하철역']

In [208]:
bt_up_1 = bus_train_num_1[up_col].groupby('지하철역').sum()
bt_down_1 = bus_train_num_1[down_col].groupby('지하철역').sum()
bt_up_2 = bus_train_num_2[up_col].groupby('지하철역').sum()
bt_down_2 = bus_train_num_2[down_col].groupby('지하철역').sum()
bt_up_3 = bus_train_num_3[up_col].groupby('지하철역').sum()
bt_down_3 = bus_train_num_3[down_col].groupby('지하철역').sum()
bt_up_4 = bus_train_num_4[up_col_].groupby('지하철역').sum()
bt_down_4 = bus_train_num_4[down_col_].groupby('지하철역').sum()

In [268]:
bt_up_1

,00시승차총승객수,1시승차총승객수,2시승차총승객수,3시승차총승객수,4시승차총승객수,5시승차총승객수,6시승차총승객수,7시승차총승객수,8시승차총승객수,9시승차총승객수,...,14시승차총승객수,15시승차총승객수,16시승차총승객수,17시승차총승객수,18시승차총승객수,19시승차총승객수,20시승차총승객수,21시승차총승객수,22시승차총승객수,23시승차총승객수
지하철역,,,,,,,,,,,,,,,,,,,,,
가락시장,4009,1689,836,537,3264,4079,6996,13505,19278,14109,...,13293,14827,15846,18207,25983,17806,13343,13116,12545,7147
가산디지털단지,5047,669,359,519,14968,35287,59151,145018,209733,126678,...,88236,98686,108376,151978,217531,157519,105230,99844,70172,36963
강남,25712,8456,3924,3807,4923,11997,28105,66026,109548,84315,...,71312,76671,83220,104037,138785,99446,76672,84964,93257,52899
강남구청,2944,15,0,0,1066,3213,7896,20622,50088,42831,...,25039,27136,28847,34584,47741,36993,25521,24955,21193,10559
강동,2703,667,434,354,1882,3223,6193,12290,17387,10806,...,8407,9441,10029,12171,15003,11170,8367,8054,7767,5575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
홍제,6309,729,374,272,7202,21928,43096,97250,124196,92257,...,84428,87930,93687,102695,117623,90373,67093,58872,50889,29544
화곡,6573,883,314,207,9660,19373,32332,69051,88938,57358,...,42974,44983,48115,53520,65678,56895,39939,37360,32351,18921
화랑대(서울여대입구),1457,0,0,0,1061,3329,6976,14755,19968,13337,...,11238,11781,12984,15259,16794,14083,9630,8084,7933,5301


In [269]:
bt_up_4

,00시승차총승객수,1시승차총승객수,2시승차총승객수,3시승차총승객수,4시승차총승객수,5시승차총승객수,6시승차총승객수,7시승차총승객수,8시승차총승객수,9시승차총승객수,...,14시승차총승객수,15시승차총승객수,16시승차총승객수,17시승차총승객수,18시승차총승객수,19시승차총승객수,20시승차총승객수,21시승차총승객수,22시승차총승객수,23시승차총승객수
지하철역,,,,,,,,,,,,,,,,,,,,,
가락시장,5449,1847,846,442,2558,5643,9243,14287,25554,17175,...,12413,13581,15347,17154,19921,16040,14254,13867,12422,7999
가산디지털단지,10610,1073,290,283,7937,29512,56815,119934,205125,133109,...,85563,94968,100772,119958,147788,130404,86529,77958,65590,39492
강남,10661,3409,2373,1904,2625,17614,24593,56904,93978,89735,...,61349,61781,65815,81808,103060,81962,50169,42236,39209,20933
강남구청,3355,78,0,0,995,6526,8910,17583,39992,38765,...,23424,24823,25836,30907,38831,30625,20853,20348,19670,9868
강동,3295,1060,250,305,1069,4997,7411,11494,17327,12013,...,10029,11200,12551,14814,17297,13026,9692,9457,8701,5925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
홍제,9738,747,270,239,2209,12412,30672,59605,89630,69137,...,75076,81557,89099,92229,102104,89730,69084,61783,57724,34835
화곡,10135,1427,616,429,4065,14118,25570,46652,65450,46488,...,45037,52344,57477,61165,73092,65344,48196,46630,43129,26252
화랑대(서울여대입구),2964,98,0,0,1694,10007,15969,32407,41551,26890,...,22558,24413,27363,30129,33780,27434,19714,17651,18194,9095


In [270]:
Seoul_junggu = ['서대문', '충정로', '시청', '회현', '서울역', '을지로입구', '명동', '을지로3가', '을지로4가', '충무로', '동대입구', '약수', '버티고개', '청구', '신당', '동대문역사문화공원', ]
Seoul_nowon = ['수락산', '마들', '노원', '중계', '하계', '월계', '광운대', '석계', '태릉입구', '화랑대', '공릉', '상계', '당고개']

### 지하철 월별시간별

In [271]:
train_total = pd.read_csv('서울시 지하철 호선별 역별 시간대별 승하차 인원 정보.csv', encoding='cp949')
train_total

,사용월,호선명,지하철역,04시-05시 승차인원,04시-05시 하차인원,05시-06시 승차인원,05시-06시 하차인원,06시-07시 승차인원,06시-07시 하차인원,07시-08시 승차인원,...,23시-24시 하차인원,00시-01시 승차인원,00시-01시 하차인원,01시-02시 승차인원,01시-02시 하차인원,02시-03시 승차인원,02시-03시 하차인원,03시-04시 승차인원,03시-04시 하차인원,작업일자
0,202007,1호선,동대문,562,16,14394,2302,9547,7086,14555,...,11620,26,898,0,0,0,0,0,0,20200803
1,202007,1호선,동묘앞,113,0,3425,884,3243,4204,6159,...,3577,8,1569,0,0,0,0,0,0,20200803
2,202007,1호선,서울역,768,12,9171,6464,12289,38755,34963,...,13089,22,1393,0,1,1,1,0,0,20200803
3,202007,1호선,시청,32,3,1730,3815,3149,19122,6256,...,2077,2,99,0,1,0,0,0,0,20200803
4,202007,1호선,신설동,233,5,9543,2426,9091,8767,18512,...,8153,12,560,0,0,0,0,0,0,20200803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38683,201501,중앙선,운길산,0,0,177,56,642,403,1292,...,633,3,222,0,0,0,0,0,0,20151223
38684,201501,중앙선,원덕,1,0,292,4,415,46,537,...,383,3,157,0,0,0,0,0,0,20151223
38685,201501,중앙선,중랑,14,1,4929,424,8539,2084,22160,...,6501,86,1617,0,0,0,0,0,0,20151223
38686,201501,중앙선,팔당,0,0,161,50,368,149,1227,...,336,8,85,0,0,0,0,0,0,20151223


In [272]:
del train_total['작업일자']
del train_total['호선명']
train_total = train_total.groupby(['사용월', '지하철역']).sum()
train_total# = train_total.reset_index()
# train_total

04시-05시 승차인원  04시-05시 하차인원  05시-06시 승차인원  05시-06시 하차인원  \
사용월    지하철역                                                                
201501 가능                  161             2          8064          1238   
       가락시장                 21             0          4428          2207   
       가산디지털단지            1184             4         13862          5119   
       가양                   10             0         11860           702   
       가좌                    0             0          1074           114   
...                        ...           ...           ...           ...   
202007 회기                  940            23         11897          3501   
       회룡                 1108            37         14162          2395   
       회현(남대문시장)            37             0          1806           779   
       효창공원앞                 6             0          3035           919   
       흑석(중앙대입구)             5             0          3438           967   

                  06시-07시 승차인원  06시-07시 하차인원  07시-08시 승차인원  07시-08시 하차인원  \
사용월    지하철역                                                                
201501 가능                13558          4480         27814          4842   
       가락시장              11179         12672         29014         19988   
       가산디지털단지           20271         33713         36407        177324   
       가양                27860          6728         80099         13778   
       가좌                 2403          2213          5704          2201   
...                        ...           ...           ...           ...   
202007 회기                21322         13349         57925         27971   
       회룡                26846          5761         57972          7883   
       회현(남대문시장)          3249         19118          4622         53353   
       효창공원앞              7064          6781         20535         12028   
       흑석(중앙대입구)          5903          5183         16663         10502   

                  08시-09시 승차인원  08시-09시 하차인원  ...  23시-24시 승차인원  23시-24시 하차인원  \
사용월    지하철역                                   ...                               
201501 가능                26713         13965  ...          1950         10191   
       가락시장              36239         61905  ...          8718         14137   
       가산디지털단지           38829        616679  ...         37281         17363   
       가양                77530         36946  ...          5470         28136   
       가좌                 7436          3761  ...           566          1672   
...                        ...           ...  ...           ...           ...   
202007 회기                69878         46842  ...         10713         24457   
       회룡                46250         11797  ...          1574         11297   
       회현(남대문시장)          7319        132902  ...          3565          2186   
       효창공원앞             32388         38278  ...          2602          6943   
       흑석(중앙대입구)         21877         14978  ...          3255          7902   

                  00시-01시 승차인원  00시-01시 하차인원  01시-02시 승차인원  01시-02시 하차인원  \
사용월    지하철역                                                                
201501 가능                  164          2672             0             0   
       가락시장               1435          5227             6           579   
       가산디지털단지            3367          4239             0             0   
       가양                  956          9033             1           706   
       가좌                   15           375             0             0   
...                        ...           ...           ...           ...   
202007 회기                  413          6252             0             0   
       회룡                  142          4258             3             2   
       회현(남대문시장)             2           308             2             2   
       효창공원앞                52          1171             0             0   
       흑석(중앙대입구)             1            19             0             0  

In [273]:
train_num_1 = train_total.loc[202002]
train_num_2 = train_total.loc[202003]
train_num_3 = train_total.loc[202004]
train_num_4 = train_total.loc[202005]

In [274]:
train_num_1 = train_num_1.reset_index()
train_num_2 = train_num_2.reset_index()
train_num_3 = train_num_3.reset_index()
train_num_4 = train_num_4.reset_index()

In [275]:
up_col = ['지하철역', '23시-24시 승차인원',
       '00시-01시 승차인원', '01시-02시 승차인원',
       '02시-03시 승차인원', '03시-04시 승차인원',
          '04시-05시 승차인원', '05시-06시 승차인원',
       '06시-07시 승차인원', '07시-08시 승차인원',
       '08시-09시 승차인원', '09시-10시 승차인원',
       '10시-11시 승차인원', '11시-12시 승차인원',
       '12시-13시 승차인원', '13시-14시 승차인원',
       '14시-15시 승차인원', '15시-16시 승차인원',
       '16시-17시 승차인원', '17시-18시 승차인원',
       '18시-19시 승차인원', '19시-20시 승차인원',
       '20시-21시 승차인원', '21시-22시 승차인원',
       '22시-23시 승차인원' ]

In [276]:
down_col = ['지하철역', '23시-24시 하차인원', '00시-01시 하차인원',
       '01시-02시 하차인원', '02시-03시 하차인원',
       '03시-04시 하차인원','04시-05시 하차인원',
       '05시-06시 하차인원', '06시-07시 하차인원',
       '07시-08시 하차인원', '08시-09시 하차인원',
       '09시-10시 하차인원', '10시-11시 하차인원',
       '11시-12시 하차인원', '12시-13시 하차인원',
       '13시-14시 하차인원', '14시-15시 하차인원',
       '15시-16시 하차인원', '16시-17시 하차인원',
       '17시-18시 하차인원', '18시-19시 하차인원',
       '19시-20시 하차인원', '20시-21시 하차인원',
       '21시-22시 하차인원', '22시-23시 하차인원']

In [277]:
train_up_1 = train_num_1[up_col]
train_down_1 = train_num_1[down_col]
train_up_2 = train_num_2[up_col]
train_down_2 = train_num_2[down_col]
train_up_3 = train_num_3[up_col]
train_down_3 = train_num_3[down_col]
train_up_4 = train_num_4[up_col]
train_down_4 = train_num_4[down_col]


train_up_1.columns = ['지하철역','00시승차총승객수', '1시승차총승객수', '2시승차총승객수', '3시승차총승객수', '4시승차총승객수', '5시승차총승객수',
       '6시승차총승객수', '7시승차총승객수', '8시승차총승객수', '9시승차총승객수', '10시승차총승객수',
       '11시승차총승객수', '12시승차총승객수', '13시승차총승객수', '14시승차총승객수', '15시승차총승객수',
       '16시승차총승객수', '17시승차총승객수', '18시승차총승객수', '19시승차총승객수', '20시승차총승객수',
       '21시승차총승객수', '22시승차총승객수', '23시승차총승객수']
train_down_1.columns = ['지하철역', '00시하차총승객수', '1시하차총승객수', '2시하차총승객수', '3시하차총승객수', '4시하차총승객수', '5시하차총승객수',
       '6시하차총승객수', '7시하차총승객수', '8시하차총승객수', '9시하차총승객수', '10시하차총승객수',
       '11시하차총승객수', '12시하차총승객수', '13시하차총승객수', '14시하차총승객수', '15시하차총승객수',
       '16시하차총승객수', '17시하차총승객수', '18시하차총승객수', '19시하차총승객수', '20시하차총승객수',
       '21시하차총승객수', '22시하차총승객수', '23시하차총승객수']
train_up_2.columns = ['지하철역','00시승차총승객수', '1시승차총승객수', '2시승차총승객수', '3시승차총승객수', '4시승차총승객수', '5시승차총승객수',
       '6시승차총승객수', '7시승차총승객수', '8시승차총승객수', '9시승차총승객수', '10시승차총승객수',
       '11시승차총승객수', '12시승차총승객수', '13시승차총승객수', '14시승차총승객수', '15시승차총승객수',
       '16시승차총승객수', '17시승차총승객수', '18시승차총승객수', '19시승차총승객수', '20시승차총승객수',
       '21시승차총승객수', '22시승차총승객수', '23시승차총승객수']
train_down_2.columns = ['지하철역', '00시하차총승객수', '1시하차총승객수', '2시하차총승객수', '3시하차총승객수', '4시하차총승객수', '5시하차총승객수',
       '6시하차총승객수', '7시하차총승객수', '8시하차총승객수', '9시하차총승객수', '10시하차총승객수',
       '11시하차총승객수', '12시하차총승객수', '13시하차총승객수', '14시하차총승객수', '15시하차총승객수',
       '16시하차총승객수', '17시하차총승객수', '18시하차총승객수', '19시하차총승객수', '20시하차총승객수',
       '21시하차총승객수', '22시하차총승객수', '23시하차총승객수']
train_up_3.columns = ['지하철역','00시승차총승객수', '1시승차총승객수', '2시승차총승객수', '3시승차총승객수', '4시승차총승객수', '5시승차총승객수',
       '6시승차총승객수', '7시승차총승객수', '8시승차총승객수', '9시승차총승객수', '10시승차총승객수',
       '11시승차총승객수', '12시승차총승객수', '13시승차총승객수', '14시승차총승객수', '15시승차총승객수',
       '16시승차총승객수', '17시승차총승객수', '18시승차총승객수', '19시승차총승객수', '20시승차총승객수',
       '21시승차총승객수', '22시승차총승객수', '23시승차총승객수']
train_down_3.columns = ['지하철역', '00시하차총승객수', '1시하차총승객수', '2시하차총승객수', '3시하차총승객수', '4시하차총승객수', '5시하차총승객수',
       '6시하차총승객수', '7시하차총승객수', '8시하차총승객수', '9시하차총승객수', '10시하차총승객수',
       '11시하차총승객수', '12시하차총승객수', '13시하차총승객수', '14시하차총승객수', '15시하차총승객수',
       '16시하차총승객수', '17시하차총승객수', '18시하차총승객수', '19시하차총승객수', '20시하차총승객수',
       '21시하차총승객수', '22시하차총승객수', '23시하차총승객수']
train_up_4.columns = ['지하철역','00시승차총승객수', '1시승차총승객수', '2시승차총승객수', '3시승차총승객수', '4시승차총승객수', '5시승차총승객수',
       '6시승차총승객수', '7시승차총승객수', '8시승차총승객수', '9시승차총승객수', '10시승차총승객수',
       '11시승차총승객수', '12시승차총승객수', '13시승차총승객수', '14시승차총승객수', '15시승차총승객수',
       '16시승차총승객수', '17시승차총승객수', '18시승차총승객수', '19시승차총승객수', '20시승차총승객수',
       '21시승차총승객수', '22시승차총승객수', '23시승차총승객수']
train_down_4.columns = ['지하철역', '00시하차총승객수', '1시하차총승객수', '2시하차총승객수', '3시하차총승객수', '4시하차총승객수', '5시하차총승객수',
       '6시하차총승객수', '7시하차총승객수', '8시하차총승객수', '9시하차총승객수', '10시하차총승객수',
       '11시하차총승객수', '12시하차총승객수', '13시하차총승객수', '14시하차총승객수', '15시하차총승객수',
       '16시하차총승객수', '17시하차총승객수', '18시하차총승객수', '19시하차총승객수', '20시하차총승객수',
       '21시하차총승객수', '22시하차총승객수', '23시하차총승객수']

In [278]:
train_up_1 = train_up_1.set_index('지하철역')
train_up_2 = train_up_2.set_index('지하철역')
train_up_3 = train_up_3.set_index('지하철역')
train_up_4 = train_up_4.set_index('지하철역')
train_down_1 = train_down_1.set_index('지하철역')
train_down_2 = train_down_2.set_index('지하철역')
train_down_3 = train_down_3.set_index('지하철역')
train_down_4 = train_down_4.set_index('지하철역')

In [279]:
train_up_1

,00시승차총승객수,1시승차총승객수,2시승차총승객수,3시승차총승객수,4시승차총승객수,5시승차총승객수,6시승차총승객수,7시승차총승객수,8시승차총승객수,9시승차총승객수,...,14시승차총승객수,15시승차총승객수,16시승차총승객수,17시승차총승객수,18시승차총승객수,19시승차총승객수,20시승차총승객수,21시승차총승객수,22시승차총승객수,23시승차총승객수
지하철역,,,,,,,,,,,,,,,,,,,,,
4.19민주묘지,555,107,0,1,0,5,1061,2397,6576,7479,...,4367,4096,3948,4173,4480,3461,2196,1627,1216,902
가능,1377,103,0,0,0,289,7012,11250,21224,19785,...,8576,7109,7373,6650,9202,9126,5592,4365,3373,2867
가락시장,6644,1273,1,1,0,17,4484,10313,26495,34561,...,21113,22246,24247,26093,31166,56006,24332,18988,18721,14557
가산디지털단지,20575,1797,0,0,0,1479,14147,14509,30622,33296,...,38636,42513,51278,67969,195749,425251,189621,102115,85777,47957
가양,3924,711,1,0,0,38,12661,20946,68002,71254,...,24068,22120,22599,24627,31780,40976,19791,14151,12064,9130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
회기,11723,1092,0,0,0,476,9680,18103,49847,65163,...,33375,33736,37456,39658,47991,44237,26943,21735,23023,21971
회룡,1690,234,0,0,0,764,12261,22260,53230,41728,...,16011,13116,12786,12513,12255,12174,7518,5569,4712,3934
회현(남대문시장),4821,641,0,0,0,35,2162,3276,4973,8015,...,30950,35690,42696,59302,91697,122556,49986,43561,19674,11923


In [280]:
train_down_1

,00시하차총승객수,1시하차총승객수,2시하차총승객수,3시하차총승객수,4시하차총승객수,5시하차총승객수,6시하차총승객수,7시하차총승객수,8시하차총승객수,9시하차총승객수,...,14시하차총승객수,15시하차총승객수,16시하차총승객수,17시하차총승객수,18시하차총승객수,19시하차총승객수,20시하차총승객수,21시하차총승객수,22시하차총승객수,23시하차총승객수
지하철역,,,,,,,,,,,,,,,,,,,,,
4.19민주묘지,1945,593,0,1,0,1,146,758,1263,3922,...,3696,3534,3576,3725,4452,6110,6880,3937,3136,2750
가능,6547,1425,0,0,0,3,1497,3489,4428,10663,...,6721,5293,8179,8253,12597,18193,17970,10358,9327,7649
가락시장,10033,3824,485,0,0,1,2437,9803,19656,65901,...,20666,20536,21941,22499,27703,42397,33860,20102,16842,15226
가산디지털단지,12637,2874,1,0,0,1,6959,31285,145576,538607,...,44662,39407,38945,37360,44263,52283,42097,23942,20085,17253
가양,17679,5861,148,0,0,0,1427,7735,19681,37301,...,19479,20987,23237,27493,33654,59683,59604,33534,30753,28077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
회기,23317,5875,0,0,0,24,3288,10293,22357,41924,...,30366,29408,29157,31865,43153,62649,56123,34240,34466,31103
회룡,12229,4248,0,0,0,34,1791,5077,7408,10280,...,12719,13598,17704,19417,25707,35107,39804,24894,18143,12789
회현(남대문시장),3424,1106,106,0,0,0,856,18909,47551,155704,...,33986,30872,28069,22560,18105,15729,10205,5829,7187,5764


## 준비완료. bt들이랑 train들이랑 비교하기 위한 틀 짜기

In [311]:
def make_index(df):
    df_index = list(df.index)
    return df_index

def intersect(a, b):
    return list(set(a) & set(b))

def co_index(a, b):
    b1 = make_index(a)
    t1 = make_index(b)
    bt_1 = intersect(b1, t1)
    return bt_1

In [348]:
#공통역을 리스트로 잡아주기
co1 = co_index(bt_up_1, train_up_1)
co2 = co_index(bt_up_2, train_up_2)
co3 = co_index(bt_up_3, train_up_3)
co4 = co_index(bt_up_4, train_up_4)
co5 = co_index(bt_down_1, train_down_1)
co6 = co_index(bt_down_2, train_down_2)
co7 = co_index(bt_down_3, train_down_3)
co8 = co_index(bt_down_4, train_down_4)

# 2월, 3월, 4월, 5월의 지하철-버스 월별시간별 승하차인원 corr비교하기

### 승차: 0.7087505633382282

In [374]:
df_1 = pd.concat([bt_up_1.loc[co1], train_up_1.loc[co1]]).T
corr_mean1=[]
for i in co1:
    corr_mean1.append(df_1.loc[:][i].corr().iloc[0][1])
cor1 = np.array(corr_mean1).mean()
cor1 

0.7326515974740277

In [375]:
df_2 = pd.concat([bt_up_2.loc[co2], train_up_2.loc[co2]]).T
corr_mean2=[]
for i in co2:
    corr_mean2.append(df_2.loc[:][i].corr().iloc[0][1])
cor2 = np.array(corr_mean2).mean()
cor2

0.7217942925085801

In [376]:
df_3 = pd.concat([bt_up_3.loc[co3], train_up_3.loc[co3]]).T
corr_mean3=[]
for i in co3:
    corr_mean3.append(df_3.loc[:][i].corr().iloc[0][1])
cor3 = np.array(corr_mean3).mean()
cor3

0.7381668718836318

In [377]:
df_4 = pd.concat([bt_up_4.loc[co4], train_up_4.loc[co4]]).T
corr_mean4=[]
for i in co4:
    corr_mean4.append(df_4.loc[:][i].corr().iloc[0][1])
cor4 = np.array(corr_mean4).mean()
cor4

0.6423894914866727

In [378]:
(cor1+cor2+cor3+cor4)/4

0.7087505633382282

### 하차: 0.5810847135258318

In [379]:
df_5 = pd.concat([bt_down_1.loc[co5], train_down_1.loc[co5]]).T
corr_mean5=[]
for i in co5:
    corr_mean5.append(df_5.loc[:][i].corr().iloc[0][1])
cor5 = np.array(corr_mean5).mean()
cor5

0.6599233098716857

In [380]:
df_6 = pd.concat([bt_down_2.loc[co6], train_down_2.loc[co6]]).T
corr_mean6=[]
for i in co6:
    corr_mean6.append(df_6.loc[:][i].corr().iloc[0][1])
cor6 = np.array(corr_mean6).mean()
cor6

0.6606826440813615

In [381]:
df_7 = pd.concat([bt_down_3.loc[co7], train_down_3.loc[co7]]).T
corr_mean7=[]
for i in co7:
    corr_mean7.append(df_7.loc[:][i].corr().iloc[0][1])
cor7 = np.array(corr_mean7).mean()
cor7

0.6797895854902987

In [382]:
df_8 = pd.concat([bt_down_4.loc[co8], train_down_4.loc[co8]]).T
corr_mean8=[]
for i in co8:
    corr_mean8.append(df_8.loc[:][i].corr().iloc[0][1])
cor8 = np.array(corr_mean8).mean()
cor8

0.3239433146599811

In [383]:
(cor5+cor6+cor7+cor8)/4

0.5810847135258318

# 결론: 승차인원 시간별 유사도 0.7, 하차인원 시간별 유사도 0.58정도로 각각 0.5 이상의 강한 상관관계를 가진다고 볼 수 있다.

## 아래는 볼필요 X

## 총 승하차 인원이 날짜별로 나와있다. 버스랑 비슷하게 승하차하는지 구해볼 수 있을 듯.

In [177]:
train_1 = pd.read_csv('CARD_SUBWAY_MONTH_202002.csv', index_col=1, encoding='cp949')
train_2 = pd.read_csv('CARD_SUBWAY_MONTH_202003.csv', index_col=1, encoding='cp949')
train_3 = pd.read_csv('CARD_SUBWAY_MONTH_202004.csv', index_col=1, encoding='cp949')
train_4 = pd.read_csv('CARD_SUBWAY_MONTH_202005.csv', index_col=1)


In [178]:
train_1

,사용일자,역명,승차총승객수,하차총승객수,등록일자
노선명,,,,,
1호선,20200201,시청,13966.0,14315.0,20200204
1호선,20200201,종각,24014.0,23628.0,20200204
1호선,20200201,종로3가,25613.0,24969.0,20200204
1호선,20200201,종로5가,19922.0,19765.0,20200204
1호선,20200201,동대문,11346.0,11210.0,20200204
...,...,...,...,...,...
5호선,20200229,신금호,2661.0,2615.0,20200303
5호선,20200229,행당,3217.0,2744.0,20200303
5호선,20200229,왕십리(성동구청),2175.0,2623.0,20200303


In [179]:
train_2

,사용일자,역명,승차총승객수,하차총승객수,등록일자
노선명,,,,,
공항철도 1호선,20200301,운서,3621.0,3795.0,20200304
우이신설선,20200301,신설동,636.0,646.0,20200304
우이신설선,20200301,보문,564.0,542.0,20200304
우이신설선,20200301,성신여대입구(돈암),1170.0,1388.0,20200304
우이신설선,20200301,정릉,1454.0,1253.0,20200304
...,...,...,...,...,...
9호선2~3단계,20200331,봉은사,11506.0,11670.0,20200403
9호선2~3단계,20200331,종합운동장,3252.0,3232.0,20200403
9호선2~3단계,20200331,삼전,5226.0,5149.0,20200403


In [180]:
train_3

,사용일자,역명,승차총승객수,하차총승객수,등록일자
노선명,,,,,
장항선,20200401,아산,1557.0,1426.0,20200404
1호선,20200401,종각,31105.0,30397.0,20200404
1호선,20200401,종로3가,21991.0,20842.0,20200404
1호선,20200401,종로5가,18925.0,19126.0,20200404
1호선,20200401,동대문,9482.0,9373.0,20200404
...,...,...,...,...,...
7호선,20200430,상동,7730.0,7786.0,20200503
7호선,20200430,삼산체육관,2616.0,2286.0,20200503
7호선,20200430,굴포천,4990.0,5027.0,20200503


In [181]:
train_4

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자
중앙선,20200501,운길산,1349,1358,20200504,NaN
중앙선,20200501,팔당,1492,1712,20200504,NaN
중앙선,20200501,도심,1691,1462,20200504,NaN
중앙선,20200501,덕소,4303,4233,20200504,NaN
중앙선,20200501,양정,645,599,20200504,NaN
...,...,...,...,...,...,...
2호선,20200531,도림천,408,398,20200603,NaN
2호선,20200531,신설동,2556,2589,20200603,NaN
2호선,20200531,신답,616,640,20200603,NaN
2호선,20200531,용답,1400,1420,20200603,NaN


In [104]:
del train_1['등록일자']
del train_2['등록일자']
del train_3['등록일자']
del train_4['하차총승객수']
del train_4['등록일자']

In [105]:
train_col = train_1.columns  

In [106]:
train_4.columns = train_col

In [107]:
train_4

,사용일자,역명,승차총승객수,하차총승객수
중앙선,20200501,운길산,1349,1358
중앙선,20200501,팔당,1492,1712
중앙선,20200501,도심,1691,1462
중앙선,20200501,덕소,4303,4233
중앙선,20200501,양정,645,599
...,...,...,...,...
2호선,20200531,도림천,408,398
2호선,20200531,신설동,2556,2589
2호선,20200531,신답,616,640
2호선,20200531,용답,1400,1420


In [114]:
train_1 = train_1.groupby(['역명','사용일자']).sum()
train_2 = train_2.groupby(['역명','사용일자']).sum()
train_3 = train_3.groupby(['역명','사용일자']).sum()
train_4 = train_4.groupby(['역명','사용일자']).sum()

In [115]:
train_1 = train_1.reset_index()
train_2 = train_2.reset_index()
train_3 = train_3.reset_index()
train_4 = train_4.reset_index()

In [116]:
train_1

,역명,사용일자,승차총승객수,하차총승객수
0,4.19민주묘지,20200201,2026.0,1715.0
1,4.19민주묘지,20200202,1674.0,1397.0
2,4.19민주묘지,20200203,2672.0,2592.0
3,4.19민주묘지,20200204,2636.0,2598.0
4,4.19민주묘지,20200205,2448.0,2407.0
...,...,...,...,...
14756,흑석(중앙대입구),20200225,5779.0,6144.0
14757,흑석(중앙대입구),20200226,5850.0,6152.0
14758,흑석(중앙대입구),20200227,5730.0,6048.0
14759,흑석(중앙대입구),20200228,5507.0,5722.0
